In [1]:
from IPython.core.display import HTML
import numpy as np


HTML("<style>.container { width:96% !important; }</style>")

# testing

In [2]:
%%capture
import matplotlib
matplotlib.use('Qt4Agg')
from matplotlib import pyplot as plt

In [ ]:
#Navigation information
'''
0: Number of Counts
1: Phase *
2: Emission * 
3: Incidence  *
4: Local Emission  
5: Local Incidence  
6: Latitude
7: Longitude  
8: Pixel Resolution  
9: Line Resolution  
10: Sample Resolution  
11: Detector Resolution  
12: North Azimuth
13: Sun Azimuth  
14: Spacecraft Azmituh  
15: Off Nadir Angle
16: Subspacecraft Ground Azimuth  
17: Subsolar Ground Azimuth
18: Morphology = Pixel Resolution/cos(Emission Angle)
19: Albedo = PixelResolution * [(1 / cos(EmissionAngle)) + (1 / cos(IncidenceAngle))]
20: Right Ascension
21: Declination
22: Body Fixed Coordinate X
23: Body Fixed Coordinate Y
24: Body Fixed Coordinate Z
'''


Numpy arrays, slice with tuple

Data: 256 bands, value of each pixel in 2D array  
Label: description of cube  
Nav: 25 measurement types, navigation info of each pixel in 2D array

2 micron window = band 50-85


In [ ]:
import pysis
data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/jupyter_folder/2005359T100445_2005360T101919/CM_1514281143_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/jupyter_folder/2005359T100445_2005360T101919/CM_1514281143_1_ir.cub').label
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/jupyter_folder/2005359T100445_2005360T101919/CM_1514281143_1_ir_nav.cub').data

#print(data) 
#print(lbl)
#type(nav)
print(nav[7,:,:])

In [ ]:
from matplotlib import pyplot as plt
#All values below vmin are colored black, reflectivity
#cmap: coloring option greyscale
plt.imshow(nav[7,:,:],vmin = -90, cmap='Greys_r') 


#Show image object
plt.show()

In [ ]:
#Band numbers and corresponding wavelengths in microns
vims_wave = np.loadtxt('/home/alanyu/Dropbox (MIT)/VIMS_UROP/vims_wave.txt') 
print(type(vims_wave))
print(vims_wave)

In [ ]:
for i in range(64):
    if nav[7,40,i]<0:
        continue
    #Plot reflectivity as a function of wavelength for a constant row value in the array:
    #plt.plot(vims_wave,data[:,40,i],label='%0.1f'%nav[7,40,i])https://isis.astrogeology.usgs.gov/documents/InstallGuide/index.html
    
    #Plot reflectivity as a function of wavelength in the 2 micron window for a constant row value in the array
    plt.plot(vims_wave[50:85],data[50:85,40,i],label='%0.1f'%nav[7,40,i])

#Each line in the legend is the varying nav[n] value (n=7, longitude)
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
plt.imshow(nav[7])

plt.show()

In [ ]:
np.shape(vims_wave)

In [ ]:
help(plt.show)

In [ ]:
len(nav[24,1,:])

In [ ]:
def constant_value_nav(nav, nav_number,value):
    """
    Given navigation data, an index into nav measurements, and a value, returns a set of all (i,j) with that value in nav. 
    """
    constants = []
    for i in range(64):
        for j in range(64):
           # print(i)
            if round(nav[nav_number,i,j]) == value:
                #print(i)
                constants.append((i,j))
            else:
                continue
                
    return constants



1) All wavelengths for fixed lon, varying lat    
    
2) only two micron window

In [ ]:
constant_longitude = constant_value_nav(nav, 7,312)
print(constant_longitude)
%matplotlib inline
for lat in constant_longitude:
    plt.plot(vims_wave, data[:,lat[0],lat[1]], label = '%0.1f'%nav[6,lat[0],lat[1]])
    
plt.legend()
plt.show()

#only two micron window
for lat in constant_longitude:
    plt.plot(vims_wave[50:85], data[50:85,lat[0],lat[1]], label = '%0.1f'%nav[6,lat[0],lat[1]])
    
plt.legend()
plt.show()

1) All wavelengths for fixed lat, varying lon   
   
2) Only two micron window

In [ ]:
constant_latitude = constant_value_nav(nav, 6,0)

print(constant_latitude)

for lon in constant_latitude:
    plt.plot(vims_wave, data[:,lon[0],lon[1]], label = '%0.1f'%nav[7,lon[0],lon[1]])
    
plt.legend()
plt.show()

#only two micron window
for lon in constant_latitude:
    plt.plot(vims_wave[50:85], data[50:85,lon[0],lon[1]], label = '%0.1f'%nav[7,lon[0],lon[1]])

plt.legend()
#plt.show()

In [3]:
import pysis
import numpy as np
import scipy.optimize as sco
from matplotlib import pyplot as plt

#Now make a function of a gaussian distribution
def gaussian(x,mean,sigma,A):
    
    return A*np.exp( -((x-mean)**2.) / (2.*sigma**2.) )


vims_wave = np.loadtxt('/home/alanyu/Dropbox (MIT)/VIMS_UROP/vims_wave.txt')
#data = pysis.CubeFile('/home/alanyu/jupyter_folder/2005359T100445_2005360T101919/CM_1514281143_1_ir.cub').data

#constant_latitude = constant_value_nav(nav,6,0)

band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 

def powerlaw(x,a, k, c):
    
    return a*x**k + c



#fit,cov = sco.curve_fit(gaussian, vims_wave[50:85],data[50:85,53,40],p0=(2,.1,.1))


/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


In [ ]:
for lon in constant_latitude:
    print(lon)
    fit,cov = sco.curve_fit(gaussian, vims_wave[50:85],data[50:85,lon[0],lon[1]],p0=(2,.1,.1))
    plt.plot(vims_wave, data[:,lon[0],lon[1]])
    plt.plot(vims_wave,gaussian(vims_wave, *fit))
    
plt.show()

In [4]:
#everything together
#normal
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').label
data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').data
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841286170_1_ir_nav.cub').data

#documentation
#least squares, residual

#filter: real amp at channel 69-70 > .01 (too dark)
#loop through directory of directories
%timeit fit,cov = sco.curve_fit(gaussian, vims_wave[50:85],data[50:85,15,30],p0=(2.08,.125,.001),maxfev=100000)

# print(fit)
# plt.plot(vims_wave,data[:,15,30])
# plt.show()
# plt.plot(vims_wave, gaussian(vims_wave,*fit))
# plt.show()
# print(vims_wave)
# band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 

# fit2, cov2 = sco.curve_fit(powerlaw, vims_wave[band_channels], data[band_channels, 15,30], p0 = (.05,-2,0), maxfev = 10000)
# v = powerlaw(vims_wave,*fit2)
# data2 = data[:,15,30]-v

# plt.plot(vims_wave, data2)
# #plt.show()

# #add gaussian
# fit3, cov3 = sco.curve_fit(gaussian, vims_wave[64:85], data2[64:85], p0 = (2.08,.125,.1), maxfev = 10000)
# plt.plot(vims_wave, gaussian(vims_wave,*fit3))
# plt.show()

27.1 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%matplotlib qt

#normal
fit,cov = sco.curve_fit(gaussian, vims_wave[50:85],data[50:85,27,38],p0=(2.08,.125,.1))
plt.plot(vims_wave,data[:,27,38])
plt.plot(vims_wave, gaussian(vims_wave,*fit))
plt.show()

# #add powerlaw ax^k + c
# band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 

# fit2, cov2 = sco.curve_fit(powerlaw, vims_wave[band_channels], data[band_channels, 27,38], p0 = (.05,-2,0), maxfev = 10000)
# v = powerlaw(vims_wave,*fit2)
# data2 = data[:,27,38]-v

# plt.plot(vims_wave, data2)
# plt.show()


# fit3, cov3 = sco.curve_fit(gaussian, vims_wave[64:85], data2[64:85], p0 = (2.08,.125,.1), maxfev = 10000)
# plt.plot(vims_wave, gaussian(vims_wave,*fit3))
# plt.plot(vims_wave,data[:,27,38])


# plt.plot(vims_wave,powerlaw(vims_wave,*fit2))
# plt.plot(vims_wave,gaussian(vims_wave,*fit))
# plt.plot(vims_wave[band_channels], data[band_channels, 27, 38], '.')
# plt.show()

In [ ]:
#add powerlaw ax^k + c
band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 

fit2, cov2 = sco.curve_fit(powerlaw, vims_wave[band_channels], data[band_channels, 27,38], p0 = (.05,-2,0), maxfev = 10000)
v = powerlaw(vims_wave,*fit2)
data2 = data[:,27,38]-v
print(v)
plt.plot(vims_wave, data2)
plt.show()

#add gaussian
fit3, cov3 = sco.curve_fit(gaussian, vims_wave[64:85], data2[64:85], p0 = (2.08,.125,.1), maxfev = 10000)
plt.plot(vims_wave, gaussian(vims_wave,*fit3))
plt.show()

In [ ]:
import pysis
import numpy as np
import scipy.optimize as sco
vims_wave = np.loadtxt('/home/alanyu/jupyter_folder/vims_wave.txt') 
data = pysis.CubeFile('/home/alanyu/jupyter_folder/2005359T100445_2005360T101919/CM_1514281143_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/jupyter_folder/2005359T100445_2005360T101919/CM_1514281143_1_ir.cub').label
nav = pysis.CubeFile('/home/alanyu/jupyter_folder/2005359T100445_2005360T101919/CM_1514281143_1_ir_nav.cub').data



In [ ]:
def create_gaussian_array(cube):
    means = np.zeros((64,64))
    sigmas = np.zeros((64,64))
    amps = np.zeros((64,64))
    for i in range(64):
        for j in range(64):
            
            if cube[69,i,j] < 0.01:
                #not on disk of Titan
                continue
            else:
                temp_data = cube[:,i,j]
                #Fit power law
                p_fit, p_cov = sco.curve_fit(powerlaw, vims_wave[band_channels], cube[band_channels, i,j], p0 = (.05,-2,0), maxfev = 10000)
                p = powerlaw(vims_wave, *p_fit)
                
                #Subtract power law
                new_data = cube[:,i,j] - p
                
                #Fit gaussian
                fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],cube[64:85,i,j],p0=(2.08,.125,.1))
                means[i,j] = fit[0]
                sigmas[i,j] = fit[1]
                amps[i,j] = fit[2]
                
    
    return (means,sigmas,amps)

In [ ]:
def generate(array):
    for i in range(64):
        yield ('Row ' + str(i), array[i,:])
generator = generate(create_gaussian_array(data)[2])

In [ ]:
largest = 0
for row in generator:
    print(next(generator))


In [ ]:
%matplotlib qt
A = create_gaussian_array(data)[1]
plt.imshow(A, vmin = 0, vmax = .2)
plt.colorbar()
np.max(A)

In [ ]:
nav[7,10]

In [ ]:
smallest_lon = None
largest_lon = None
for i in range(64):
    for j in range(64):
        if nav[7,i,j] >= 0 and (smallest_lon is None or nav[7,i,j] < smallest_lon):
            smallest_lon=nav[7,i,j]
        if largest_lon is None or nav[7,i,j] > largest_lon:
            largest_lon = nav[7,i,j]
            
smallest_lat = None
largest_lat = None
for i in range(64):
    for j in range(64):
        if nav[6,i,j] >= 0 and (smallest_lat is None or nav[6,i,j] < smallest_lat):
            smallest_lat=nav[6,i,j]
        if largest_lat is None or nav[6,i,j] > largest_lat:
            largest_lat = nav[6,i,j]

In [ ]:
print(largest_lat)
print(smallest_lat)

print(largest_lon)
print(smallest_lon)

In [ ]:
import pysis

data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/jupyter_folder/2016127T223449_2016130T200342/CM_1841283690_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/jupyter_folder/2016127T223449_2016130T200342/CM_1841283690_1_ir.cub').label
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/jupyter_folder/2016127T223449_2016130T200342/CM_1841283690_1_ir_nav.cub').data
length = lbl['IsisCube']['Instrument']['SwathLength']
width = lbl['IsisCube']['Instrument']['SwathWidth']
for i in range(width):
    if nav[7,40,i]<0:
        continue
    #Plot reflectivity as a function of wavelength for a constant row value in the array:
    #plt.plot(vims_wave,data[:,40,i],label='%0.1f'%nav[7,40,i])https://isis.astrogeology.usgs.gov/documents/InstallGuide/index.html
    
    #Plot reflectivity as a function of wavelength in the 2 micron window for a constant row value in the array
    plt.plot(vims_wave[50:85],data[50:85,40,i],label='%0.1f'%nav[7,40,i])
    
plt.show()